In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
# We're creating two RDD, one is from the README file of Spark
# and the other is directly from a list within the notebook.
#
# If you downloaded Spark, the README file is in the same folder
# as the one you extracted. If you use other package management
# methods like 'brew', 'dnf', 'apt', etc. you will need to figure
# out the path of Spark by printing sys.path.

rdd = sc.textFile('/Users/rachellim/CUSP/BDM/Lab6/README.md')
testRdd = sc.parallelize([1,2,3,4,5])

In [3]:
# Here, we're showing the difference between map() and flatMap()
# doing the line split and get back the first 3 elements, take(3).
# - map() is a one-to-one mapping, just like Python, so the first
# line prints out 3 lists, each consists words per each.
# - flatMap() is a one-to-many mapping, like MapReduce's map(). So
# the second line prints out only 3 words.

wordsPerLine = rdd.map(lambda line: line.split()).take(3)
words = rdd.flatMap(lambda line: line.split()).take(3)

print ('%s\n\n%s' % (wordsPerLine, words))

[[u'#CDS_DataViz']]

[u'#CDS_DataViz']


In [4]:
# This is the word count example with Spark using the approach
# shown in the slides, i.e. staying true to the MapReduce paradigm.
# Note that groupByKey() will sort and group everything together by
# keys first. Then the function in mapValues() will each get applied
# per each (key, list of values) pair. This could be an issue if we
# have a pair with lots of values since all of the values have to be
# stored in memory.

wc = rdd.flatMap(lambda line: line.split()) \
        .map(lambda x: (x.lower(), 1)) \
        .groupByKey() \
        .mapValues(lambda values: sum(values))
wc.take(2)

[(u'#cds_dataviz', 1)]

In [5]:
# This is another approach with reduceByKey() instead of groupByKey().
# The reduce function provided for reduceByKey() only takes 2 params
# at a time, thus, doesn't suffer the scalability issue. It also has
# better benefits in term of parallelism.

wc = rdd.flatMap(lambda line: line.split()) \
        .map(lambda x: (x.lower(), 1)) \
        .reduceByKey(lambda x,y: x+y)
wc.take(2)

[(u'#cds_dataviz', 1)]

In [6]:
# If we'd like to compute the top 3 most popular words in Spark. We
# can use the RDD's top() function directly. This is much easier
# than the two-step MapReduce job, where we had to first compute the
# top 3 words per partition, then another top 3 on top of that. In
# fact, this is exactly how Spark RDD's top() function is implemented.
# More info can be found here:
# https://github.com/apache/spark/blob/master/python/pyspark/rdd.py#L1249
wc.top(3, key=lambda x: x[1])

[(u'#cds_dataviz', 1)]

## LAB 6 - Task 1

In [7]:
SAT_FN = 'SAT_Results.csv'
HSD_FN = 'DOE_High_School_Directory_2014-2015.csv'

In [8]:
# We read the SAT score to our RDD. Note that the use_unicode can be
# changed accordingly to your data file to handle Unicode. If you cannot
# parse your data due to an 'utf8' or 'ascii' decoding issue, it might
# be a good thing to try flipping the use_unicode parameter here.

sat = sc.textFile(SAT_FN, use_unicode=False).cache()

# This line for us to list the column index and column names to see
# which column we need to use for our task. In this case, we're
# interested in the number of test takers (#2) and the math score (#4).
list(enumerate(sat.first().split(',')))

[(0, 'DBN'),
 (1, 'SCHOOL NAME'),
 (2, 'Num of SAT Test Takers'),
 (3, 'SAT Critical Reading Avg. Score'),
 (4, 'SAT Math Avg. Score'),
 (5, 'SAT Writing Avg. Score')]

In [9]:
# Note that, our data input includes a header line that we don't want to
# use in analysis. We can remove the header line from our RDD by doing
# a 'filter' to remove all rows that matches the header like below. Though
# this works, it means that we have to apply the filter function on *all*
# row, which could be a lot of computation.

noHeaderRDD = sat.filter(lambda x: not x.startswith('DBN,SCHOOL'))
print (sat.first())
print (noHeaderRDD.first())

DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score
02M047,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECONDARY SCHOOL,16,395,400,387


In [10]:
# Alternatively, we can perform the header checking per-partition, instead
# of per-row like below. mapPartitions() is another type of map operators
# in Spark that is similar to Hadoop Streaming's map(). It is many-to-many.
# RDD in Spark are divided into partitions (as we read or as provided by
# HDFS), each partition can be processed in parallel using a function
# supplied to the mapPartitions() call.
# 
# In addition to mapPartitions(), Spark also provides a variation called
# mapPartitionsWithIndex() that provides information on which partition
# we are currently processing. Indeed, mapPartitionsWithIndex() is the
# the operator with the lowest overhead (since mapPartitions() get mapped
# to mapPartitionsWithIndex) and also the most efficient one among all the
# map operators.
#
# So our logic below is to use the partition index to check if we're hitting
# the header (aka the first partition). If so, we just skip the first row.

def extractScores(partId, records):
    if partId==0:
        records.next()
    import csv
    reader = csv.reader(records)
    for row in reader:
        if row[2]!='s': # to filter our bad-quality data
            (dbn,takers,score) = (row[0], int(row[2]), int(row[4]))
            yield (dbn, (score*takers, takers))

satScores = sat.mapPartitionsWithIndex(extractScores)
satScores.take(5)

[('02M047', (6400, 16)),
 ('21K410', (207575, 475)),
 ('30Q301', (43120, 98)),
 ('17K382', (22066, 59)),
 ('18K637', (13335, 35))]

In [11]:
# Here we do the same thing with the school directory data
schools = sc.textFile(HSD_FN, use_unicode=False).cache()
list(enumerate(schools.first().split(',')))

[(0, 'dbn'),
 (1, 'school_name'),
 (2, 'boro'),
 (3, 'building_code'),
 (4, 'phone_number'),
 (5, 'fax_number'),
 (6, 'grade_span_min'),
 (7, 'grade_span_max'),
 (8, 'expgrade_span_min'),
 (9, 'expgrade_span_max'),
 (10, 'bus'),
 (11, 'subway'),
 (12, 'primary_address_line_1'),
 (13, 'city'),
 (14, 'state_code'),
 (15, 'zip'),
 (16, 'website'),
 (17, 'total_students'),
 (18, 'campus_name'),
 (19, 'school_type'),
 (20, 'overview_paragraph'),
 (21, 'program_highlights'),
 (22, 'language_classes'),
 (23, 'advancedplacement_courses'),
 (24, 'online_ap_courses'),
 (25, 'online_language_courses'),
 (26, 'extracurricular_activities'),
 (27, 'psal_sports_boys'),
 (28, 'psal_sports_girls'),
 (29, 'psal_sports_coed'),
 (30, 'school_sports'),
 (31, 'partner_cbo'),
 (32, 'partner_hospital'),
 (33, 'partner_highered'),
 (34, 'partner_cultural'),
 (35, 'partner_nonprofit'),
 (36, 'partner_corporate'),
 (37, 'partner_financial'),
 (38, 'partner_other'),
 (39, 'addtl_info1'),
 (40, 'addtl_info2'),
 (4

In [12]:
def extractSchools(partId, list_of_records):
    if partId==0: 
        list_of_records.next() # skipping the first line
    import csv
    reader = csv.reader(list_of_records)
    for row in reader:
        if len(row)==58 and row[17].isdigit():
            (dbn, boro, total_students) = (row[0], row[2], int(row[17]))
            if total_students>500: # filter to keep the large schools
                yield (dbn, boro)

largeSchools = schools.mapPartitionsWithIndex(extractSchools)

In [13]:
scores = largeSchools.join(satScores).values() \
    .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])) \
    .mapValues(lambda x: x[0]/x[1]) \
    .collect()

In [14]:
scores

[('Bronx', 470),
 ('Manhattan', 514),
 ('Brooklyn', 487),
 ('Staten Island', 477),
 ('Queens', 474)]

In [15]:
buses = "B39, M14A, M14D, M15, M15-SBS, M21, M9"
subways = "B, D to Grand St; F to East Broadway ; J, M, Z to Delancey St-Essex St"

In [16]:
lines = buses.split(', ')
lines = subways.split(' ; ')[1].split(' to ')[0].split(', ')
lines

['J', 'M', 'Z']

In [38]:
rowLen = len(schools.first().split(','))

def extractSchools2(partId, list_of_records):
    if partId==0: 
        next(list_of_records) # skipping the first line
    import csv
    reader = csv.reader(list_of_records)
    for row in reader:
        if len(row)==rowLen:
            (dbn, buses, subways) = (row[0], row[10], row[11])
            lines = buses.split(', ') + ', '.join([direction.split(' to' )[0]
                                                  for direction in subways.split(' ; ')]).split(', ') 
    yield (dbn, lines)

schoolsWithLines = schools.mapPartitionsWithIndex(extractSchools2)

In [39]:
schoolsWithLines.join(satScores).take(3)

[('32K556',
  (['B38', 'B46', 'B47', 'B52', 'B54', 'B60', 'Q24', 'J', 'M', 'Z'],
   (8234, 23))),
 ('12X550',
  (['Bx11', 'Bx27', 'Bx36', 'Bx4', 'Bx4A', 'Q44', '6'], (13566, 42)))]

In [40]:
schoolsWithLines.join(satScores) \
    .flatMap(lambda dbn_stat: [(line, dbn_stat[1][1]) for line in dbn_stat[1][0]]) \
    .reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1])) \
    .mapValues(lambda x: int(x[0]/x[1])) \
    .sortBy(lambda x:-x[1])\
    .take(10)

[('B52', 358),
 ('M', 358),
 ('B38', 358),
 ('Q24', 358),
 ('B47', 358),
 ('B54', 358),
 ('J', 358),
 ('B46', 358),
 ('B60', 358),
 ('Z', 358)]

In [41]:
rowLen = len(schools.first().split(','))

def findS1115(partId, list_of_records):
    if partId==0: 
        next(list_of_records) # skipping the first line
    import csv
    reader = csv.reader(list_of_records)
    for row in reader:
        if len(row)==rowLen:
            (dbn, buses, subways) = (row[0], row[10], row[11])
            if 'S1115' in buses:
                yield buses

In [42]:
schools.mapPartitionsWithIndex(findS1115).collect()

['M15, M15-SBS, M20, M5, M9, S1115',
 'M20, M21, M5, S1115',
 'M15, M15-SBS, M20, M5, M9, S1115',
 'M103, M15, M15-SBS, M20, M22, M5, M9, S1115',
 'M20, M22, M5, M9, S1115',
 'M103, M15, M15-SBS, M20, M22, M5, M9, S1115',
 'M15, M15-SBS, M20, M5, M9, S1115',
 'M20, M21, M5, S1115']

In [28]:
dfSchools = spark.read.load(HSD_FN, format='csv', header=True, inferSchema=True)
dfSchools = dfSchools.na.drop(subset=['boro'])
dfSchools = dfSchools.filter(dfSchools['total_students']>500)
dfSchools = dfSchools.select('dbn', 'boro')
dfSchools.count()

176

In [29]:
dfSchools.show()

+------+---------+
|   dbn|     boro|
+------+---------+
|01M450|Manhattan|
|01M539|Manhattan|
|01M696|Manhattan|
|02M374|Manhattan|
|02M400|Manhattan|
|02M408|Manhattan|
|02M412|Manhattan|
|02M413|Manhattan|
|02M416|Manhattan|
|02M418|Manhattan|
|02M420|Manhattan|
|02M425|Manhattan|
|02M475|Manhattan|
|02M489|Manhattan|
|02M519|Manhattan|
|02M520|Manhattan|
|02M529|Manhattan|
|02M542|Manhattan|
|02M580|Manhattan|
|02M600|Manhattan|
+------+---------+
only showing top 20 rows



In [32]:
#put backtick around column names if dataset column name contains . 
dfScores = spark.read.load(SAT_FN, format='csv', header=True, inferSchema=True)
dfScores = dfScores.select('DBN', dfScores['`SAT Math Avg. Score`'].cast('int').alias('score'),
                          dfScores['Num of SAT Test Takers'].cast('int').alias('ntakers')).na.drop()
dfScores1 = dfScores.select('DBN', (dfScores.score*dfScores.ntakers).alias('sum'), 'ntakers')
#dfScores.take(5)
dfScores1.show()

+------+------+-------+
|   DBN|   sum|ntakers|
+------+------+-------+
|02M047|  6400|     16|
|21K410|207575|    475|
|30Q301| 43120|     98|
|17K382| 22066|     59|
|18K637| 13335|     35|
|32K403| 18300|     50|
|09X365| 18306|     54|
|11X270| 22064|     56|
|05M367| 12078|     33|
|14K404| 24276|     68|
|30Q575| 66420|    135|
|13K336|  3366|      9|
|04M635| 17712|     48|
|24Q264| 40406|     89|
|17K408| 19494|     57|
|19K618| 22260|     60|
|27Q309| 13644|     36|
|32K552| 24388|     67|
|13K499| 26208|     72|
|07X600| 30400|     76|
+------+------+-------+
only showing top 20 rows



In [ ]:
scores = largeSchools.join(satScores).values() \
    .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])) \
    .mapValues(lambda x: x[0]/x[1]) \
    .collect()

In [34]:
dfResults = dfSchools.join(dfScores1, dfSchools.dbn==dfScores1.DBN, how='inner')
dfResults = dfResults.groupBy('boro').sum('sum', 'ntakers')
dfResults = dfResults.withColumn('avg', (dfResults[1]/dfResults[2]).cast('int'))
dfResults = dfResults.select('boro', 'avg')
dfResults.show()

+-------------+---+
|         boro|avg|
+-------------+---+
|       Queens|474|
|     Brooklyn|487|
|Staten Island|477|
|    Manhattan|514|
|        Bronx|470|
+-------------+---+

